# Principal Component Analysis versus Normal Modes with an Anisotropic Harmonic Well

In [ ]:
import openmm as mm
from openmm import app
from openmm import unit
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Definición del sistema.

n_particles = 1
mass = 100 * unit.amu

In [ ]:
# Creación del sistema.

system = mm.System()

for ii in range(n_particles):
    system.addParticle(mass)

In [ ]:
# Añadiendo el potencial externo al sistema

Kx = 5.0 * unit.kilocalorie_per_mole/unit.nanometers**2
Ky = 50.0 * unit.kilocalorie_per_mole/unit.nanometers**2
Kz = 500.0 * unit.kilocalorie_per_mole/unit.nanometers**2

force = mm.CustomExternalForce('A*x^2 + B*y^2 + C*z^2')
force.addGlobalParameter('A', Kx/2.0)
force.addGlobalParameter('B', Ky/2.0)
force.addGlobalParameter('C', Kz/2.0)

for ii in range(n_particles):
    force.addParticle(ii, [])
    
_ = system.addForce(force)

In [ ]:
# Definición del estado termodinámico y el integrador.

step_size = 0.01*unit.picoseconds
temperature = 300.0*unit.kelvin
friction = 1.0/unit.picosecond # Damping para la dinámica de Langevin

integrator = mm.LangevinIntegrator(temperature, friction, step_size)

In [ ]:
# Creación de la plataforma.

platform_name = 'CUDA' # Cambia a CUDA...
platform = mm.Platform.getPlatformByName(platform_name)

In [ ]:
# Creación del contexto.

context = mm.Context(system, integrator, platform)

In [ ]:
# Condiciones iniciales

initial_positions  = np.zeros([n_particles, 3], np.float32) * unit.nanometers
initial_velocities = np.zeros([n_particles, 3], np.float32) * unit.nanometers/unit.picoseconds

context.setPositions(initial_positions)
context.setVelocities(initial_velocities)

In [ ]:
# Parámetros de la simulación.

simulation_time = 0.10*unit.nanosecond
saving_time = 1.0*unit.picoseconds

In [ ]:
n_steps_per_period = int(saving_time/step_size) # número de pasos del periodo de guardado
n_periods = int(simulation_time/saving_time) # número de periodos guardados

In [ ]:
# Creación de arrays reporteros del tiempo, la posición y la velocidad.

times = np.zeros([n_periods], np.float32) * unit.picoseconds
positions = np.zeros([n_periods, n_particles, 3], np.float32) * unit.nanometers

In [ ]:
# Almacenamiento en reporteros de las condiciones iniciales para tiempo 0

state = context.getState(getPositions=True)

times[0] = state.getTime()
positions[0] = state.getPositions()

In [ ]:
# Ejecuto el bucle sobre el número de periodos que vamos a simular

for ii in tqdm(range(1, n_periods)):
    context.getIntegrator().step(n_steps_per_period)
    state = context.getState(getPositions=True)
    times[ii] = state.getTime()
    positions[ii] = state.getPositions()

In [ ]:
#Graficando las posiciones de cada dirección contra el tiempo

for ii, axis_label in zip(range(3),['X','Y','Z']):
    plt.plot(times, positions[:,0,ii])
    plt.ylabel('{} ({})'.format(axis_label, positions.unit))
    plt.xlabel('time ({})'.format(times.unit))
    plt.show()

In [ ]:
positions.shape

Construimos la matriz de covarianza y la diagonalizamos -hacemos el análisis de componentes principales-:


\begin{equation}
C = \begin{pmatrix}
\langle x^{2} \rangle - \langle x \rangle^{2} & \langle xy \rangle - \langle x \rangle \langle y \rangle & \langle xz \rangle - \langle x \rangle \langle z \rangle \\
\langle yx \rangle - \langle y \rangle \langle x \rangle & \langle y^{2} \rangle - \langle y \rangle^{2} & \langle yz \rangle - \langle y \rangle \langle z \rangle \\
\langle zx \rangle - \langle z \rangle \langle x \rangle & \langle zy \rangle - \langle z \rangle \langle y \rangle & \langle z^{2} \rangle - \langle z \rangle^{2} \\
\end{pmatrix}.
\end{equation}


Preguntas a responder:

- ¿Puedes calcular la distribución de probabilidad de que la partícula esté en un punto x del eje X observada de la trayectoria? (hacer histograma en el eje X y normalizarlo debidamente para representar la densidad de probabilidad).
- ¿Puedes calcular la función de densidad de probabilidad P(x) teórica? ¿Son comparables?
- ¿Puedes calcular la desviación cuadrática estandard teórica $\sigma_{x} = \langle x^{2} \rangle - \langle x \rangle^{2}$? ¿Es comparable al valor observado de la trayectoria?
- ¿Qué relación hay entre $\langle x^{2} \rangle - \langle x \rangle^{2}$ y la frecuencia natural clásica de oscilación de la partícula en el eje $X$?
- ¿Cómo son los elementos de fuera de la diagonal de la matriz de covarianza?¿Qué significa?

# Para el futuro (por terminar)
El potencial tiene los ejes de simetría alineados en X, Y y Z. Por construcción del hamiltoniano, la dinámica del sistema está decorrelacionada en los tres ejes. Ahora deberíamos de probar a construir una matriz de covarianza sencilla no alineada. Podríamos girar el pozo armónico en el espacio y correr una nueva dinámica. Pero vamos mejor a hacer un cambio de base de $[(1, 0, 0), (0, 1, 0), (0, 0, 1)]$ a $[(1, 1, 0), (1, 0, 1), (0, 1, 1)]$ (normalizado, claro).